In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt

import xgboost as xgb

from cleaning import clean_data

from sklearn.utils.class_weight import compute_sample_weight


In [2]:
X_train, Y_train, X_test, Y_test = clean_data()
sample_weights = compute_sample_weight(class_weight='balanced',y=Y_train)

In [3]:
#for ideal number of estimatos
'''model = xgb.XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 1000,
    max_depth = 5,
    min_child_weight = 1,
    gamma = 0,
    subsample = 0.8,
    colsample_bytree = 0.8,
    objective = 'multi:softmax',
    num_class = 5,
)
xgb_param = model.get_xgb_params()
cvresult = xgb.cv(xgb_param, train, num_boost_round=model.get_params()['n_estimators'], nfold=5,
            metrics='auc', early_stopping_rounds=50)
model.set_params(n_estimators=cvresult.shape[0])
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print('Confusion Matrix\n',confusion_matrix(Y_test, Y_pred))
print('Cohen Kappa', cohen_kappa_score(Y_test, Y_pred, weights='quadratic'))
print('Accuracy Score', accuracy_score(Y_test, Y_pred))
print('MSE', mean_squared_error(Y_test, Y_pred))'''

"model = xgb.XGBClassifier(\n    learning_rate = 0.1,\n    n_estimators = 1000,\n    max_depth = 5,\n    min_child_weight = 1,\n    gamma = 0,\n    subsample = 0.8,\n    colsample_bytree = 0.8,\n    objective = 'multi:softmax',\n    num_class = 5,\n)\nxgb_param = model.get_xgb_params()\ncvresult = xgb.cv(xgb_param, train, num_boost_round=model.get_params()['n_estimators'], nfold=5,\n            metrics='auc', early_stopping_rounds=50)\nmodel.set_params(n_estimators=cvresult.shape[0])\nmodel.fit(X_train, Y_train)\nY_pred = model.predict(X_test)\nprint('Confusion Matrix\n',confusion_matrix(Y_test, Y_pred))\nprint('Cohen Kappa', cohen_kappa_score(Y_test, Y_pred, weights='quadratic'))\nprint('Accuracy Score', accuracy_score(Y_test, Y_pred))\nprint('MSE', mean_squared_error(Y_test, Y_pred))"

In [4]:
#for max_depth and min_child_weight
'''hyper_params = {'max_depth':range(1,10),
                'min_child_weight':range(1,6)}
search = RandomizedSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141,
                                                        gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax'), 
                            param_distributions = hyper_params, verbose = 1, cv = 5)
search.fit(X_train, Y_train)
print(search.best_score_)
print(search.best_estimator_)'''

"hyper_params = {'max_depth':range(1,10),\n                'min_child_weight':range(1,6)}\nsearch = RandomizedSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141,\n                                                        gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax'), \n                            param_distributions = hyper_params, verbose = 1, cv = 5)\nsearch.fit(X_train, Y_train)\nprint(search.best_score_)\nprint(search.best_estimator_)"

In [5]:
#for subsample and colsample_bytree
'''hyper_params = {'subsample':[i/10.0 for i in range(6,10)],
                'colsample_bytree':[i/10.0 for i in range(6,10)]
                }
search = RandomizedSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141, max_depth=5, min_child_weight=3, 
                                                        gamma=0,objective= 'multi:softmax'), 
                            param_distributions = hyper_params, verbose = 1, cv = 5)
search.fit(X_train, Y_train)
print(search.best_score_)
print(search.best_estimator_)
'''

"hyper_params = {'subsample':[i/10.0 for i in range(6,10)],\n                'colsample_bytree':[i/10.0 for i in range(6,10)]\n                }\nsearch = RandomizedSearchCV(estimator=xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141, max_depth=5, min_child_weight=3, \n                                                        gamma=0,objective= 'multi:softmax'), \n                            param_distributions = hyper_params, verbose = 1, cv = 5)\nsearch.fit(X_train, Y_train)\nprint(search.best_score_)\nprint(search.best_estimator_)\n"

In [6]:
#with sample_weights
model = xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141, max_depth=5,
                            min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax')
model.fit(X_train, Y_train, sample_weight=sample_weights)
Y_pred = model.predict(X_test)
print('Confusion Matrix\n',confusion_matrix(Y_test, Y_pred))
print('Cohen Kappa', cohen_kappa_score(Y_test, Y_pred, weights='quadratic'))
print('Train Accuracy Score', accuracy_score(Y_train, model.predict(X_train)))
print('Test Accuracy Score', accuracy_score(Y_test, Y_pred))

Confusion Matrix
 [[ 17  48  13  12  37]
 [ 87 381 133 129 177]
 [ 68 363 191 288 337]
 [ 55 216 116 270 302]
 [ 40 166 104 146 802]]
Cohen Kappa 0.3235483169589657
Train Accuracy Score 0.4971891376846117
Test Accuracy Score 0.36927523343708313


In [7]:
#without sample_weights
model = xgb.XGBClassifier(learning_rate = 0.1, n_estimators=141, max_depth=5,
                            min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'multi:softmax')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print('Confusion Matrix\n',confusion_matrix(Y_test, Y_pred))
print('Cohen Kappa', cohen_kappa_score(Y_test, Y_pred, weights='quadratic'))
print('Train Accuracy Score', accuracy_score(Y_train, model.predict(X_train)))
print('Test Accuracy Score', accuracy_score(Y_test, Y_pred))

Confusion Matrix
 [[  0  42  35   7  43]
 [  0 268 329  76 234]
 [  2 213 471 172 389]
 [  1 129 273 183 373]
 [  1  97 214  71 875]]
Cohen Kappa 0.3196618557002461
Train Accuracy Score 0.518913768461172
Test Accuracy Score 0.39951089373054693
